In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType,IntegerType,FloatType

# Create a SparkSession
spark = SparkSession.builder.appName("Marvel_heroes").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/20 14:45:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
schema = StructType([ \
        StructField("id",IntegerType(),False), \
        StructField("name",StringType(),False)])
names = spark.read.schema(schema).option("sep"," ").csv("../../data/Marvel_Names")
names.show()

+---+--------------------+
| id|                name|
+---+--------------------+
|  1|24-HOUR MAN/EMMANUEL|
|  2|3-D MAN/CHARLES CHAN|
|  3|    4-D MAN/MERCURIO|
|  4|             8-BALL/|
|  5|                   A|
|  6|               A'YIN|
|  7|        ABBOTT, JACK|
|  8|             ABCISSA|
|  9|                ABEL|
| 10|ABOMINATION/EMIL BLO|
| 11|ABOMINATION | MUTANT|
| 12|         ABOMINATRIX|
| 13|             ABRAXAS|
| 14|          ADAM 3,031|
| 15|             ABSALOM|
| 16|ABSORBING MAN/CARL C|
| 17|ABSORBING MAN | MUTA|
| 18|                ACBA|
| 19|ACHEBE, REVEREND DOC|
| 20|            ACHILLES|
+---+--------------------+
only showing top 20 rows



In [22]:
lines = spark.read.text("../../data/Marvel_Graph")
connections = lines \
    .withColumn("id",func.split(func.col("value")," ")[0]) \
    .withColumn("connections", func.size(func.split(func.col("value")," ")) -1) \
    .groupBy('id').agg(func.sum('connections').alias("connections"))
connections.show()

+----+-----------+
|  id|connections|
+----+-----------+
| 691|          7|
|1159|         12|
|3959|        143|
|1572|         36|
|2294|         15|
|1090|          5|
|3606|        172|
|3414|          8|
| 296|         18|
|4821|         17|
|2162|         42|
|1436|         10|
|1512|         12|
|6194|         15|
|6240|         12|
| 829|         38|
|2136|          7|
|5645|         21|
|2069|        264|
| 467|          1|
+----+-----------+
only showing top 20 rows



In [19]:
mostPopular = connections.sort(func.desc('connections')).first()
mostPopularName = names.filter(names.id == mostPopular[0]).select('name').first()
print(mostPopularName.name + " with: " + str(mostPopular.connections) + " connections")

CAPTAIN AMERICA with: 1937 connections


In [52]:
minNumOfConnections = connections.agg(func.min("connections")).first()[0]
mostObscure = connections.filter(connections.connections == minNumOfConnections)
mostObscureNames = mostObscure.join(names, "id")
mostObscureNames.show()

+----+-----------+--------------------+
|  id|connections|                name|
+----+-----------+--------------------+
| 467|          1|        BERSERKER II|
| 577|          1|              BLARE/|
|3490|          1|MARVEL BOY II/MARTIN|
|3489|          1|MARVEL BOY/MARTIN BU|
|2139|          1|      GIURESCU, RADU|
|1089|          1|       CLUMSY FOULUP|
|1841|          1|              FENRIS|
|4517|          1|              RANDAK|
|5028|          1|           SHARKSKIN|
| 835|          1|     CALLAHAN, DANNY|
|1408|          1|         DEATHCHARGE|
|4784|          1|                RUNE|
|4945|          1|         SEA LEOPARD|
|4602|          1|         RED WOLF II|
|6411|          1|              ZANTOR|
|3014|          1|JOHNSON, LYNDON BAIN|
|3298|          1|          LUNATIK II|
|2911|          1|                KULL|
|2117|          1|GERVASE, LADY ALYSSA|
+----+-----------+--------------------+



CAPTAIN AMERICA with: 1937 connections
